In [ ]:
from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
import dotenv
import os
from sqlalchemy import create_engine
import urllib

dotenv.load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

server = 'optimus.database.windows.net'
database = 'Neutrino'
username = 'Slav'
password = 'Stevan_13' 
driver= 'ODBC Driver 17 for SQL Server'

# Create the connection URL
params = urllib.parse.quote_plus(
    'DRIVER={' + driver + '};'
    'SERVER=' + server + ';'
    'DATABASE=' + database + ';'
    'UID=' + username + ';'
    'PWD=' + password
)

# Create the engine
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(params))
db = engine.connect()
    
llm = OpenAI(api_key= api_key, temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.chains import create_sql_query_chain
from langchain.chat_models import ChatOpenAI
import dotenv
import os
from sqlalchemy import create_engine
import urllib

dotenv.load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

server = 'optimus.database.windows.net'
database = 'Neutrino'
username = 'Slav'
password = 'Stevan_13' 
driver = 'ODBC Driver 17 for SQL Server'

# Create the connection URL
params = urllib.parse.quote_plus(
    'DRIVER={' + driver + '};'
    'SERVER=' + server + ';'
    'DATABASE=' + database + ';'
    'UID=' + username + ';'
    'PWD=' + password
)

# Create the engine
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(params))

# Create an instance of SQLDatabase
sql_db = SQLDatabase(engine)

# Initialize OpenAI
llm = OpenAI(api_key=api_key, temperature=0, verbose=True)

# Create SQLDatabaseChain
chain = create_sql_query_chain(ChatOpenAI(temperature=0), sql_db)
response = chain.invoke({"question": "How many employees are there"})
print(response)


In [ ]:
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.chains import create_sql_query_chain
from langchain.chat_models import ChatOpenAI
import dotenv
import os
from sqlalchemy import create_engine
import urllib

dotenv.load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

server = 'optimus.database.windows.net'
database = 'Neutrino'
username = 'Slav'
password = 'Stevan_13' 
driver = 'ODBC Driver 17 for SQL Server'

# Create the connection URL
params = urllib.parse.quote_plus(
    'DRIVER={' + driver + '};'
    'SERVER=' + server + ';'
    'DATABASE=' + database + ';'
    'UID=' + username + ';'
    'PWD=' + password
)

# Create the engine
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(params))

# Create an instance of SQLDatabase
sql_db = SQLDatabase(engine)

# Initialize OpenAI
llm = OpenAI(api_key=api_key, temperature=0, verbose=True)

# Create SQLDatabaseChain with the correct SQLDatabase instance
db_chain = SQLDatabaseChain.from_llm(llm, sql_db, verbose=True)

# Run query using db_chain
response = db_chain.run("Considering that Order revenue is a sum of part reveneu of the order, which order has the highest revenue?")
print(response)


In [1]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
# from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
import dotenv
import os
from sqlalchemy import create_engine
import urllib

dotenv.load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

server = 'optimus.database.windows.net'
database = 'Neutrino'
username = 'Slav'
password = 'Stevan_13' 
driver = 'ODBC Driver 17 for SQL Server'

# Create the connection URL
params = urllib.parse.quote_plus(
    'DRIVER={' + driver + '};'
    'SERVER=' + server + ';'
    'DATABASE=' + database + ';'
    'UID=' + username + ';'
    'PWD=' + password
)
# Create the engine
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(params))
# Create an instance of SQLDatabase
db = SQLDatabase(engine)

agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0)),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)
agent_executor.run(
    "Create forecasting based on date and revenue using python"
)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Orders, TestData
Thought: I should query the schema of the Orders table
Action: sql_db_schema
Action Input: Orders
Observation: 
CREATE TABLE [Orders] (
	[Date] VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[OrderNumber] VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[CustomerName] VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[Revenue] FLOAT(53) NULL, 
	[PartCost] FLOAT(53) NULL, 
	[PartNumber] VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL
)

/*
3 rows from Orders table:
Date	OrderNumber	CustomerName	Revenue	PartCost	PartNumber
7/13/2021	ORD10001	Customer_118	1245.25	553.99	PART193
7/13/2021	ORD10001	Customer_118	1284.77	396.69	PART10
7/13/2021	ORD10001	Customer_118	1794.37	964.39	PART88
*/
Thought: I should query the database for the date and revenue
Action: sql_db_query
Action Input: SELECT Date, Revenue FROM Orders ORDER BY Revenue DESC LIMIT 10

'The top 10 dates and revenues are: 6/4/2020, 1996.19; 7/22/2020, 1991.07; 11/20/2021, 1976.0; 11/11/2020, 1971.71; 1/19/2021, 1966.89; 1/24/2021, 1963.1; 11/20/2021, 1957.41; 4/21/2020, 1955.85; 2/27/2021, 1955.17; 5/24/2022, 1952.07.'